<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#How-to-Run-it-on-SBB-data" data-toc-modified-id="How-to-Run-it-on-SBB-data-1">How to Run it on SBB data</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-2">Results</a></span></li></ul></div>

# How to Run it on SBB data

In [1]:
from run_csa_sbb import run, all_stations, connections_sbb

csa loading...
csa_utils loading...
run_csa_sbb loading...


In [6]:
departure_timestamp = 1515396840

csa_sbb = run(
    departure_station='Zürich HB',
    arrival_station='Zürich, ETH/Universitätsspital',
    departure_timestamp = departure_timestamp,
    min_certainty = 0.05,
    speed = 4,
    top_n = 3
)

# Results

In [10]:
import numpy as np
import pandas as pd
import datetime

In [13]:
np.random.seed(3)

for s in np.random.choice(['Zürich, ETH/Universitätsspital'], size = 1):
    print()
    print('{} -> {}'.format(csa_sbb.dep_station, s))
    print()
    for i, path in enumerate(csa_sbb.get_paths(s)):
        first_non_walking_edges = [i for i, e in enumerate(path.edges()) if e['trip_id'] != 'walking']
        start_index = 0 if len(first_non_walking_edges) == 0 else first_non_walking_edges[0]
        path_duration = path.edges()[-1]['arrival_ts'] - path.edges()[start_index]['arrival_ts']
        path_duration += path.edges()[start_index]['duration']
        if start_index > 0:
            path_duration += path.edges()[0]['duration']
        path_duration = int(path_duration / 60 * 1000) / 1000
        path_certainty = int(path.last()['cum_certainty'] * 1000) / 1000
        path_starting_time = datetime.datetime.fromtimestamp(path.edges()[start_index]['departure_ts']).strftime('%H:%M')
        path_name = "[#{} at {}] in {} minutes with {} certainty".format(i+1, path_starting_time, path_duration, path_certainty)
        print()
        lines = [['FROM', 'TO', 'DURATION', 'CERTAINTY', 'TRIP_ID']]
        for e in path.edges():
            line = [e['departure_station'], e['arrival_station'], int(e['duration'] / 60 * 100) / 100 , int(e['certainty'] * 100) / 100, e['trip_id']]
            line = [str(e) for e in line]
            lines.append(line)
        col_widths = [max([len(word) for word in row]) + 2 for row in pd.DataFrame(lines).T.values ]  # padding
        for row in lines:
            print(''.join(word.ljust(col_widths[i]) for i, word in enumerate(row)))
        print()


Zürich HB -> Zürich, ETH/Universitätsspital


FROM               TO                              DURATION  CERTAINTY  TRIP_ID                
Zürich HB          Zürich, Haldenegg               5.46      1.0        walking                
Zürich, Haldenegg  Zürich, ETH/Universitätsspital  2.0       1.0        85:3849:81666-02006-1  



In [6]:
from csa import CSA

In [7]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [10]:
%lprun -f CSA.add_connection run(\
    departure_station='Zürich HB',\
    arrival_station='Zürich, Zoo',\
    departure_timestamp = departure_timestamp,\
    min_certainty = 0.05,\
    speed = 5,\
    top_n = 5\
)

/usr/local/lib/python3.5/dist-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
